In [1]:
from dask.distributed import Client, LocalCluster, performance_report
import dask.dataframe as dd
import os
import pandas as pd
import numpy as np
import pickle
import json
import sys
from stellargraph import StellarGraph, IndexedArray
from stellargraph.data import UniformRandomMetaPathWalk
from gensim.models import Word2Vec
from p_tqdm import p_umap, p_imap, p_map
from tqdm import tqdm
from scipy import sparse
from itertools import combinations, product
from functools import partial
from sklearn.preprocessing import OneHotEncoder, MultiLabelBinarizer
from sklearn.utils.extmath import cartesian
import csv

In [2]:
os.chdir('/home/rcgonzal/DSC180Malware/m2v-adversarial-hindroid/')
!pwd

/home/rcgonzal/DSC180Malware/m2v-adversarial-hindroid


In [3]:
from src.data.hindroid import *

%load_ext autoreload
%autoreload 2

In [4]:
outfolder = "data/out/miniset-train"
num_apis = 2547149

In [5]:
api_sets_large = pd.read_pickle('data/temp/package_api_sets.pkl')
api_sets_small = api_sets_large[api_sets_large.apply(len)<=1000]
api_sets_large = api_sets_large[api_sets_large.apply(len)>1000]
len(api_sets_small), len(api_sets_large)

(22337, 254)

In [6]:
def build_P_mat(api_sets, num_apis):
    comb_func = lambda api_list: np.array(list(combinations(api_list, r=2)))
    row = []
    col = []
    for combos in p_imap(comb_func, api_sets):
        row.extend(combos[:,0])
        col.extend(combos[:,1])
    mat = sparse.csr_matrix(([True]*len(row), (row, col)), shape=(num_apis, num_apis), dtype=bool)
    del row, col
    return mat

  0%|          | 0/22337 [00:00<?, ?it/s]

808932515


In [10]:
%time A_mat, B_mat, P_mat = build_matrices('data/out/test-sample')

Dask Cluster: LocalCluster('tcp://127.0.0.1:33587', workers=4, threads=8, memory=68.72 GB)
Dashboard port: 8787
Constructing A matrix...
Constructing B matrix (Step 1)...


  0%|          | 0/16790 [00:00<?, ?it/s]

1it [00:00,  7.15it/s]

Constructing B matrix (Step 2)...
Constructed: <18746x18746 sparse matrix of type '<class 'numpy.float64'>'
	with 435082 stored elements in Compressed Sparse Row format>


Constructing P matrix...


/home/rcgonzal/.local/lib/python3.8/site-packages/scipy/sparse/_index.py:125: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


  0%|          | 0/255 [00:00<?, ?it/s]

Constructed: <18746x18746 sparse matrix of type '<class 'numpy.bool_'>'
	with 6293344 stored elements in Compressed Sparse Row format>
CPU times: user 27.5 s, sys: 2.57 s, total: 30.1 s
Wall time: 39.2 s


In [11]:
A_mat, B_mat, P_mat

(<6x18746 sparse matrix of type '<class 'numpy.int64'>'
 	with 22446 stored elements in Compressed Sparse Row format>,
 <18746x18746 sparse matrix of type '<class 'numpy.float64'>'
 	with 435082 stored elements in Compressed Sparse Row format>,
 <18746x18746 sparse matrix of type '<class 'numpy.bool_'>'
 	with 6293344 stored elements in Compressed Sparse Row format>)

In [21]:
%time make_models('data/out/test-sample')

Fitting AAT model...
Fitting ABAT model...
Fitting APAT model...
Fitting ABPBTAT model...
Fitting APBPTAT model...
              acc  recall        f1
kernel                             
AAT      0.833333     1.0  0.909091
ABAT     0.833333     1.0  0.909091
APAT     0.833333     1.0  0.909091
ABPBTAT  0.833333     1.0  0.909091
APBPTAT  0.833333     1.0  0.909091
CPU times: user 155 ms, sys: 17.1 ms, total: 172 ms
Wall time: 201 ms


In [6]:
%time build_matrices(outfolder)

Dask Cluster: LocalCluster('tcp://127.0.0.1:42865', workers=4, threads=8, memory=68.72 GB)
Dashboard port: 8787
Constructing A matrix...
Constructing B matrix (Step 1)...


  0%|          | 0/1590622 [00:00<?, ?it/s]

/home/rcgonzal/.local/lib/python3.8/site-packages/scipy/sparse/_index.py:125: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


Constructed: <2547149x2547149 sparse matrix of type '<class 'numpy.bool_'>'
	with 92465515 stored elements in Compressed Sparse Row format>
Constructing P matrix...


  0%|          | 0/196958 [00:00<?, ?it/s]

/home/rcgonzal/.local/lib/python3.8/site-packages/scipy/sparse/_index.py:125: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


Constructed: <2547149x2547149 sparse matrix of type '<class 'numpy.bool_'>'
	with 276226513 stored elements in Compressed Sparse Row format>
CPU times: user 50min 34s, sys: 4min 27s, total: 55min 2s
Wall time: 1h 14min 32s


(<448x2547149 sparse matrix of type '<class 'numpy.int64'>'
 	with 8167935 stored elements in Compressed Sparse Row format>,
 <2547149x2547149 sparse matrix of type '<class 'numpy.bool_'>'
 	with 92465515 stored elements in Compressed Sparse Row format>,
 <2547149x2547149 sparse matrix of type '<class 'numpy.bool_'>'
 	with 276226513 stored elements in Compressed Sparse Row format>)

In [5]:
%time build_matrices('data/out/all-apps')

Dask Cluster: LocalCluster('tcp://127.0.0.1:46077', workers=4, threads=8, memory=68.72 GB)
Dashboard port: 8787
Constructing A matrix...
Constructing B matrix (Step 1)...


  0%|          | 0/3152345 [00:00<?, ?it/s]

/home/rcgonzal/.local/lib/python3.8/site-packages/scipy/sparse/_index.py:125: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


Constructed: <5160691x5160691 sparse matrix of type '<class 'numpy.bool_'>'
	with 201168077 stored elements in Compressed Sparse Row format>
Constructing P matrix...


  0%|          | 0/372844 [00:00<?, ?it/s]

/home/rcgonzal/.local/lib/python3.8/site-packages/scipy/sparse/_index.py:125: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


Constructed: <5160691x5160691 sparse matrix of type '<class 'numpy.bool_'>'
	with 538489931 stored elements in Compressed Sparse Row format>
CPU times: user 1h 44min 2s, sys: 10min 1s, total: 1h 54min 4s
Wall time: 2h 56min 18s


(<6072x5160691 sparse matrix of type '<class 'numpy.int64'>'
 	with 33609663 stored elements in Compressed Sparse Row format>,
 <5160691x5160691 sparse matrix of type '<class 'numpy.bool_'>'
 	with 201168077 stored elements in Compressed Sparse Row format>,
 <5160691x5160691 sparse matrix of type '<class 'numpy.bool_'>'
 	with 538489931 stored elements in Compressed Sparse Row format>)

In [ ]:
%time make_models('data/out/all-apps')

Fitting AAT model...
Fitting ABAT model...
Fitting APAT model...
Fitting ABPBTAT model...


In [6]:
source_folder = os.path.join('data/out/all-apps', 'hindroid')
A = sparse.load_npz(os.path.join(source_folder, 'A_mat.npz')).astype('float32')
B = sparse.load_npz(os.path.join(source_folder, 'B_mat.npz')).astype('float32').tocsr()
P = sparse.load_npz(os.path.join(source_folder, 'P_mat.npz')).astype('float32').tocsr()

In [8]:
A

<6072x5160691 sparse matrix of type '<class 'numpy.float32'>'
	with 33609663 stored elements in Compressed Sparse Row format>

In [6]:
A.shape[0]

6072

In [14]:
A[6000:6500]

<72x5160691 sparse matrix of type '<class 'numpy.uint32'>'
	with 2334201 stored elements in Compressed Sparse Row format>

In [11]:
A_ = A[0:100]

In [ ]:
ABA = A * B *  A.T
ABA

In [11]:
print(ABA.data.nbytes + ABA.indptr.nbytes + ABA.indices.nbytes)

7286804


In [5]:
api_map_path = os.path.join(outfolder, 'api_map.csv')
apis = pd.read_csv(api_map_path, index_col='api').uid.str.replace('api', '').astype(int).values

In [ ]:
P_edges_filepath = os.path.join('data', 'temp', 'P_edges.csv')

pd.DataFrame([(api, api) for api in apis], columns=['source', 'target']).to_csv(P_edges_filepath, index=False)

with open(P_edges_filepath, 'a') as P_edges_file:
    p_umap(partial(parse_P_edges, P_edges_file=P_edges_file), pd.read_pickle('data/temp/package_api_sets.pkl'))
#     for api_list in tqdm():
#         parse_P_edges(api_list, P_edges_writer)

  0%|          | 0/246035 [00:00<?, ?it/s]

In [ ]:
import sys
sys.getsizeof(temp_edges)*56

In [10]:
import sys
sys.getsizeof((1,2))

56

In [ ]:
sparse.csr

In [12]:
package_api_sets = pd.read_pickle('data/temp/package_api_sets.pkl')
pkg_lens = package_api_sets.apply(len)
pkg_lens[pkg_lens>1000]

source
package0          1716
package101        3951
package102        1201
package1029      12128
package103        1314
                 ...  
package227284     7222
package227326     2003
package227601     1189
package227630     2287
package245813     1474
Name: target, Length: 254, dtype: int64

In [27]:
package_api_sets.items()

In [ ]:
sys.getsizeof(list(combinations((i for i in range(174_318)), r=2)))

In [22]:
pkg_lens[pkg_lens>10000].index

source
package1029       12128
package119        14488
package1269       33490
package1294       25454
package133        10883
package139        17335
package141621     38315
package227134     27881
package2276       11418
package2363      106893
package251       174318
package2816       18863
package286        10675
package338        10943
package580        16923
Name: target, dtype: int64

In [23]:
pd.read_csv(os.path.join(outfolder, 'package_map.csv'), index_col='uid').package[pkg_lens[pkg_lens>10000].index]

source
package1029                              Lcom/google/protobuf
package119                         Landroidx/appcompat/widget
package1269      Lcom/google/android/gms/internal/measurement
package1294      Lcom/google/android/gms/measurement/internal
package133                             Landroidx/fragment/app
package139                      Landroidx/recyclerview/widget
package141621                                              LX
package227134                                            Lcal
package2276                             Lcom/vungle/publisher
package2363                  Lcom/google/android/gms/internal
package251               Lcom/google/android/gms/internal/ads
package2816                                                Lo
package286        Lcom/google/android/gms/common/api/internal
package338             Lcom/google/android/gms/games/internal
package580                         Landroid/support/v7/widget
Name: package, dtype: object

In [ ]:
%time make_models(outfolder)

In [45]:
pd.read_csv(os.path.join(outfolder, 'app_list.csv'))

,app,app_dir,category
0,635a57a483cda858f78f14386e76aab4,/teams/DSC180A_FA20_A00/a04malware/malware/Ban...,malware
1,912054e230f08f3747c2966d3f92944f,/teams/DSC180A_FA20_A00/a04malware/malware/You...,malware
2,29b4865171cdfad2a6f011614a1a8038,/teams/DSC180A_FA20_A00/a04malware/malware/Fak...,malware
3,2a92f33fa4b5af4e61d70eb15a28030d,/teams/DSC180A_FA20_A00/a04malware/malware/You...,malware
4,35ecbda726e1e56467bf8b0e0dbe2c2a,/teams/DSC180A_FA20_A00/a04malware/malware/Fak...,malware
5,com.sfeehha.bubble419,/teams/DSC180A_FA20_A00/a04malware/popular-app...,popular-apps


In [33]:
popular_apps = pd.read_csv('data/out/all-apps/app_list.csv', dtype=str)
popular_apps = popular_apps[popular_apps.category=='popular-apps']
popular_apps.sample(1)

,app,app_dir,category,malware
5871,com.sega.PhantasyStarII,/teams/DSC180A_FA20_A00/a04malware/popular-app...,popular-apps,0


In [25]:
apps = (
    pd.read_csv(
        os.path.join(outfolder, 'app_map.csv'),
        dtype=str,
        index_col='app'
    ).join(
        pd.read_csv(
            os.path.join(outfolder, 'app_list.csv'),
            dtype=str,
            index_col='app'
        )
    )
)
apps = apps.reset_index().set_index(apps.uid.str.replace('app', '').astype(int)).sort_index()
apps

,app,uid,app_dir,category
uid,,,,
0,635a57a483cda858f78f14386e76aab4,app0,/teams/DSC180A_FA20_A00/a04malware/malware/Ban...,malware
1,912054e230f08f3747c2966d3f92944f,app1,/teams/DSC180A_FA20_A00/a04malware/malware/You...,malware
2,29b4865171cdfad2a6f011614a1a8038,app2,/teams/DSC180A_FA20_A00/a04malware/malware/Fak...,malware
3,2a92f33fa4b5af4e61d70eb15a28030d,app3,/teams/DSC180A_FA20_A00/a04malware/malware/You...,malware
4,35ecbda726e1e56467bf8b0e0dbe2c2a,app4,/teams/DSC180A_FA20_A00/a04malware/malware/Fak...,malware


In [3]:
edges_path = os.path.join(outfolder, 'edges.csv')
api_map_path = os.path.join(outfolder, 'api_map.csv')
app_map_path = os.path.join(outfolder, 'app_map.csv')
apis = pd.read_csv(api_map_path, index_col='api').uid.str.replace('api', '').astype(int).values
apps = pd.read_csv(app_map_path, index_col='app').uid.str.replace('app', '').astype(int).values
edges_path

'../data/out/test-sample/edges.csv'

In [4]:
edges = dd.read_csv(edges_path, dtype=str)
edges['target'] = edges.target.str.replace('api', '').astype(int)
app_api_edges = edges[edges.source.str.startswith('app')]

num_apps = apps.size
num_apis = apis.size

app_api_edges['source'] = app_api_edges.source.str.replace('app', '').astype(int)

In [31]:
app_api_edges.groupby('source').target.unique().compute()
mlb = MultiLabelBinarizer(sparse_output=True)
mlb.fit([(api,) for api in apis])
A_mat = mlb.transform(app_api_edges.groupby('source').target.unique().compute().sort_index())
A_mat

<5x15875 sparse matrix of type '<class 'numpy.int64'>'
	with 18900 stored elements in Compressed Sparse Row format>

In [29]:
api_method_edges = edges[edges.source.str.startswith('method')]
api_sets = api_method_edges.groupby('source').target.unique()
B_edges = set(pd.Series(apis).apply(lambda x: (x,x)))

for combos in p_imap(partial(combinations, r=2), api_sets.compute()):
    B_edges.update(combos)
    B_edges.update([(combo[1], combo[0]) for combo in combos])
    
B_mat = mlb.transform(pd.DataFrame(B_edges).groupby(0)[1].unique().sort_index())
B_mat

  0%|          | 0/13958 [00:00<?, ?it/s]

<15875x15875 sparse matrix of type '<class 'numpy.int64'>'
	with 204206 stored elements in Compressed Sparse Row format>

In [27]:
api_package_edges = edges[edges.source.str.startswith('package')]
api_sets = api_package_edges.groupby('source').target.unique()
P_edges = set(pd.Series(apis).apply(lambda x: (x,x)))

P_edges.update([(api, api) for api in apis])

for combos in p_imap(partial(combinations, r=2), api_sets.compute()):
    P_edges.update(combos)

P_mat = mlb.transform(pd.DataFrame(P_edges).groupby(0)[1].unique().sort_index())
P_mat

  0%|          | 0/250 [00:00<?, ?it/s]

<15875x15875 sparse matrix of type '<class 'numpy.int64'>'
	with 2800987 stored elements in Compressed Sparse Row format>